# Advanced indexing

In [1]:
import sys
sys.path.insert(0, '..')
import zarr
import numpy as np
np.random.seed(42)
import cProfile
zarr.__version__

'2.1.5.dev118+dirty'

## Functionality and API

### Indexing a 1D array with a Boolean (mask) array

Supported via ``get/set_mask_selection()`` and ``.vindex[]``. Also supported via ``get/set_orthogonal_selection()`` and ``.oindex[]``.

In [2]:
a = np.arange(10)
za = zarr.array(a, chunks=2)
ix = [False,  True,  False,  True, False, True, False,  True,  False,  True]

In [3]:
# get items
za.vindex[ix]

array([1, 3, 5, 7, 9])

In [4]:
# get items
za.oindex[ix]

array([1, 3, 5, 7, 9])

In [5]:
# set items
za.vindex[ix] = a[ix] * 10
za[:]

array([ 0, 10,  2, 30,  4, 50,  6, 70,  8, 90])

In [6]:
# set items
za.oindex[ix] = a[ix] * 100
za[:]

array([  0, 100,   2, 300,   4, 500,   6, 700,   8, 900])

In [7]:
# if using .oindex, indexing array can be any array-like, e.g., Zarr array
zix = zarr.array(ix, chunks=2)
za = zarr.array(a, chunks=2)
za.oindex[zix]  # will not load all zix into memory

array([1, 3, 5, 7, 9])

### Indexing a 1D array with a 1D integer (coordinate) array

Supported via ``get/set_coordinate_selection()`` and ``.vindex[]``. Also supported via ``get/set_orthogonal_selection()`` and ``.oindex[]``.

In [8]:
a = np.arange(10)
za = zarr.array(a, chunks=2)
ix = [1, 3, 5, 7, 9]

In [9]:
# get items
za.vindex[ix]

array([1, 3, 5, 7, 9])

In [10]:
# get items
za.oindex[ix]

array([1, 3, 5, 7, 9])

In [11]:
# set items
za.vindex[ix] = a[ix] * 10
za[:]

array([ 0, 10,  2, 30,  4, 50,  6, 70,  8, 90])

In [12]:
# set items
za.oindex[ix] = a[ix] * 100
za[:]

array([  0, 100,   2, 300,   4, 500,   6, 700,   8, 900])

### Indexing a 1D array with a multi-dimensional integer (coordinate) array

Supported via ``get/set_coordinate_selection()`` and ``.vindex[]``.

In [13]:
a = np.arange(10)
za = zarr.array(a, chunks=2)
ix = np.array([[1, 3, 5], [2, 4, 6]])

In [14]:
# get items
za.vindex[ix]

array([[1, 3, 5],
       [2, 4, 6]])

In [15]:
# set items
za.vindex[ix] = a[ix] * 10
za[:]

array([ 0, 10, 20, 30, 40, 50, 60,  7,  8,  9])

### Slicing a 1D array with step <> 1

Slices with step <> 1 are supported via ``get/set_orthogonal_selection()`` and ``.oindex[]``. Internally these are converted to an integer array via ``np.arange``.

In [16]:
a = np.arange(10)
za = zarr.array(a, chunks=2)

In [17]:
# get items
za.oindex[1::2]

array([1, 3, 5, 7, 9])

In [18]:
# get items with negative step
za.oindex[-1::-2]

array([9, 7, 5, 3, 1])

In [19]:
# set items
za.oindex[1::2] = a[1::2] * 10
za[:]

array([ 0, 10,  2, 30,  4, 50,  6, 70,  8, 90])

### Orthogonal (outer) indexing of multi-dimensional arrays

Orthogonal (a.k.a. outer) indexing is supported with either Boolean or integer arrays, in combination with integers and slices. This functionality is provided via the ``get/set_orthogonal_selection()`` methods. For convenience, this functionality is also available via the ``.oindex[]`` property.

In [20]:
a = np.arange(15).reshape(5, 3)
za = zarr.array(a, chunks=(3, 2))
za[:]

array([[ 0,  1,  2],
       [ 3,  4,  5],
       [ 6,  7,  8],
       [ 9, 10, 11],
       [12, 13, 14]])

In [21]:
# orthogonal indexing with Boolean arrays
ix0 = [False, True, False, True, False]
ix1 = [True, False, True]
za.get_orthogonal_selection((ix0, ix1))

array([[ 3,  5],
       [ 9, 11]])

In [22]:
# alternative API
za.oindex[ix0, ix1]

array([[ 3,  5],
       [ 9, 11]])

In [23]:
# orthogonal indexing with integer arrays
ix0 = [1, 3]
ix1 = [0, 2]
za.get_orthogonal_selection((ix0, ix1))

array([[ 3,  5],
       [ 9, 11]])

In [24]:
# alternative API
za.oindex[ix0, ix1]

array([[ 3,  5],
       [ 9, 11]])

In [25]:
# combine with slice
za.oindex[[1,  3], :]

array([[ 3,  4,  5],
       [ 9, 10, 11]])

In [26]:
# combine with slice
za.oindex[:, [0, 2]]

array([[ 0,  2],
       [ 3,  5],
       [ 6,  8],
       [ 9, 11],
       [12, 14]])

In [27]:
# set items via Boolean selection
ix0 = [False, True, False, True, False]
ix1 = [True, False, True]
selection = ix0, ix1
value = 42
za.set_orthogonal_selection(selection, value)
za[:]

array([[ 0,  1,  2],
       [42,  4, 42],
       [ 6,  7,  8],
       [42, 10, 42],
       [12, 13, 14]])

In [28]:
# alternative API
za.oindex[ix0, ix1] = 44
za[:]

array([[ 0,  1,  2],
       [44,  4, 44],
       [ 6,  7,  8],
       [44, 10, 44],
       [12, 13, 14]])

In [29]:
# set items via integer selection
ix0 = [1, 3]
ix1 = [0, 2]
selection = ix0, ix1
value = 46
za.set_orthogonal_selection(selection, value)
za[:]

array([[ 0,  1,  2],
       [46,  4, 46],
       [ 6,  7,  8],
       [46, 10, 46],
       [12, 13, 14]])

In [30]:
# alternative API
za.oindex[ix0, ix1] = 48
za[:]

array([[ 0,  1,  2],
       [48,  4, 48],
       [ 6,  7,  8],
       [48, 10, 48],
       [12, 13, 14]])

### Coordinate indexing of multi-dimensional arrays

Selecting arbitrary points from a multi-dimensional array by indexing with integer (coordinate) arrays is supported. This functionality is provided via the ``get/set_coordinate_selection()`` methods. For convenience, this functionality is also available via the ``.vindex[]`` property.

In [31]:
a = np.arange(15).reshape(5, 3)
za = zarr.array(a, chunks=(3, 2))
za[:]

array([[ 0,  1,  2],
       [ 3,  4,  5],
       [ 6,  7,  8],
       [ 9, 10, 11],
       [12, 13, 14]])

In [32]:
# get items
ix0 = [1, 3]
ix1 = [0, 2]
za.get_coordinate_selection((ix0, ix1))

array([ 3, 11])

In [33]:
# alternative API
za.vindex[ix0, ix1]

array([ 3, 11])

In [34]:
# set items
za.set_coordinate_selection((ix0, ix1), 42)
za[:]

array([[ 0,  1,  2],
       [42,  4,  5],
       [ 6,  7,  8],
       [ 9, 10, 42],
       [12, 13, 14]])

In [35]:
# alternative API
za.vindex[ix0, ix1] = 44
za[:]

array([[ 0,  1,  2],
       [44,  4,  5],
       [ 6,  7,  8],
       [ 9, 10, 44],
       [12, 13, 14]])

### Mask indexing of multi-dimensional arrays

Selecting arbitrary points from a multi-dimensional array by a Boolean array is supported. This functionality is provided via the ``get/set_mask_selection()`` methods. For convenience, this functionality is also available via the ``.vindex[]`` property.

In [36]:
a = np.arange(15).reshape(5, 3)
za = zarr.array(a, chunks=(3, 2))
za[:]

array([[ 0,  1,  2],
       [ 3,  4,  5],
       [ 6,  7,  8],
       [ 9, 10, 11],
       [12, 13, 14]])

In [37]:
ix = np.zeros_like(a, dtype=bool)
ix[1, 0] = True
ix[3, 2] = True
za.get_mask_selection(ix)

array([ 3, 11])

In [38]:
za.vindex[ix]

array([ 3, 11])

In [39]:
za.set_mask_selection(ix, 42)
za[:]

array([[ 0,  1,  2],
       [42,  4,  5],
       [ 6,  7,  8],
       [ 9, 10, 42],
       [12, 13, 14]])

In [40]:
za.vindex[ix] = 44
za[:]

array([[ 0,  1,  2],
       [44,  4,  5],
       [ 6,  7,  8],
       [ 9, 10, 44],
       [12, 13, 14]])

### Selecting fields from arrays with a structured dtype

All ``get/set_selection_...()`` methods support a ``fields`` argument which allows retrieving/replacing data for a specific field or fields. Also h5py-like API is supported where fields can be provided within ``__getitem__``, ``.oindex[]`` and ``.vindex[]``.

In [41]:
a = np.array([(b'aaa', 1, 4.2),
              (b'bbb', 2, 8.4),
              (b'ccc', 3, 12.6)], 
             dtype=[('foo', 'S3'), ('bar', 'i4'), ('baz', 'f8')])
za = zarr.array(a, chunks=2)
za[:]

array([(b'aaa', 1,   4.2), (b'bbb', 2,   8.4), (b'ccc', 3,  12.6)],
      dtype=[('foo', 'S3'), ('bar', '<i4'), ('baz', '<f8')])

In [42]:
za['foo']

array([b'aaa', b'bbb', b'ccc'],
      dtype='|S3')

In [43]:
za['foo', 'baz']

array([(b'aaa',   4.2), (b'bbb',   8.4), (b'ccc',  12.6)],
      dtype=[('foo', 'S3'), ('baz', '<f8')])

In [44]:
za[:2, 'foo']

array([b'aaa', b'bbb'],
      dtype='|S3')

In [45]:
za[:2, 'foo', 'baz']

array([(b'aaa',  4.2), (b'bbb',  8.4)],
      dtype=[('foo', 'S3'), ('baz', '<f8')])

In [46]:
za.oindex[[0, 2], 'foo']

array([b'aaa', b'ccc'],
      dtype='|S3')

In [47]:
za.vindex[[0, 2], 'foo']

array([b'aaa', b'ccc'],
      dtype='|S3')

In [48]:
za['bar'] = 42
za[:]

array([(b'aaa', 42,   4.2), (b'bbb', 42,   8.4), (b'ccc', 42,  12.6)],
      dtype=[('foo', 'S3'), ('bar', '<i4'), ('baz', '<f8')])

In [49]:
za[:2, 'bar'] = 84
za[:]

array([(b'aaa', 84,   4.2), (b'bbb', 84,   8.4), (b'ccc', 42,  12.6)],
      dtype=[('foo', 'S3'), ('bar', '<i4'), ('baz', '<f8')])

Note that this API differs from numpy when selecting multiple fields. E.g.:

In [50]:
a['foo', 'baz']

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [51]:
a[['foo', 'baz']]

array([(b'aaa',   4.2), (b'bbb',   8.4), (b'ccc',  12.6)],
      dtype=[('foo', 'S3'), ('baz', '<f8')])

In [52]:
za['foo', 'baz']

array([(b'aaa',   4.2), (b'bbb',   8.4), (b'ccc',  12.6)],
      dtype=[('foo', 'S3'), ('baz', '<f8')])

In [53]:
za[['foo', 'baz']]

IndexError: unsupported selection type; expected integer or contiguous slice, got ['foo', 'baz']

## 1D Benchmarking

In [54]:
c = np.arange(100000000)
c.nbytes

800000000

In [55]:
%time zc = zarr.array(c)
zc.info

CPU times: user 428 ms, sys: 56 ms, total: 484 ms
Wall time: 128 ms


Type,zarr.core.Array
Data type,int64
Shape,"(100000000,)"
Chunk shape,"(97657,)"
Order,C
Read-only,False
Compressor,"Blosc(cname='lz4', clevel=5, shuffle=SHUFFLE, blocksize=0)"
Store type,builtins.dict
No. bytes,800000000 (762.9M)
No. bytes stored,11854081 (11.3M)
Storage ratio,67.5


In [61]:
%timeit c.copy()

121 ms ± 1.16 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [62]:
%timeit zc[:]

256 ms ± 7.92 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### bool dense selection

In [63]:
# relatively dense selection - 10%
ix_dense_bool = np.random.binomial(1, 0.1, size=c.shape[0]).astype(bool)
np.count_nonzero(ix_dense_bool)

9995616

In [64]:
%timeit c[ix_dense_bool]

243 ms ± 5.16 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [65]:
%timeit zc.oindex[ix_dense_bool]

426 ms ± 3.59 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [66]:
%timeit zc.vindex[ix_dense_bool]

550 ms ± 13.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [115]:
import tempfile
import cProfile
import pstats

def profile(statement, sort='time', restrictions=(7,)):
    with tempfile.NamedTemporaryFile() as f:
        cProfile.run(statement, filename=f.name)
        pstats.Stats(f.name).sort_stats(sort).print_stats(*restrictions)


In [116]:
profile('zc.oindex[ix_dense_bool]')

Wed Nov  8 11:19:16 2017    /tmp/tmp4t23nh90

         83015 function calls in 0.469 seconds

   Ordered by: internal time
   List reduced from 82 to 7 due to restriction <7>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     1025    0.196    0.000    0.196    0.000 {method 'nonzero' of 'numpy.ndarray' objects}
     1024    0.144    0.000    0.153    0.000 ../zarr/core.py:1028(_decode_chunk)
     1024    0.043    0.000    0.223    0.000 ../zarr/core.py:849(_chunk_getitem)
     1024    0.009    0.000    0.009    0.000 {built-in method numpy.core.multiarray.count_nonzero}
     1025    0.007    0.000    0.232    0.000 ../zarr/indexing.py:544(__iter__)
     1024    0.006    0.000    0.206    0.000 /home/aliman/pyenv/zarr_20171023/lib/python3.6/site-packages/numpy/lib/index_tricks.py:26(ix_)
     2048    0.005    0.000    0.005    0.000 ../zarr/core.py:337(<genexpr>)




In [102]:
profile('zc.oindex[ix_dense_bool]')

Wed Nov  8 11:14:55 2017    /tmp/tmpc6yiwbhy

         83015 function calls in 0.486 seconds

   Ordered by: internal time
   List reduced from 82 to 5 due to restriction <5>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     1025    0.195    0.000    0.195    0.000 {method 'nonzero' of 'numpy.ndarray' objects}
     1024    0.149    0.000    0.158    0.000 ../zarr/core.py:1028(_decode_chunk)
     1024    0.044    0.000    0.229    0.000 ../zarr/core.py:849(_chunk_getitem)
     1024    0.017    0.000    0.017    0.000 {built-in method numpy.core.multiarray.count_nonzero}
     1025    0.007    0.000    0.233    0.000 ../zarr/indexing.py:544(__iter__)




Method ``nonzero`` is being called internally within numpy to convert bool to int selections, no way to avoid.

In [117]:
profile('zc.vindex[ix_dense_bool]')

Wed Nov  8 11:19:20 2017    /tmp/tmpxd6eg1gj

         51354 function calls in 0.592 seconds

   Ordered by: internal time
   List reduced from 87 to 7 due to restriction <7>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.217    0.108    0.217    0.108 {method 'nonzero' of 'numpy.ndarray' objects}
     1024    0.100    0.000    0.106    0.000 ../zarr/core.py:1028(_decode_chunk)
        2    0.093    0.046    0.093    0.046 ../zarr/indexing.py:640(<genexpr>)
     1024    0.044    0.000    0.171    0.000 ../zarr/core.py:849(_chunk_getitem)
        1    0.029    0.029    0.029    0.029 {built-in method numpy.core.multiarray.ravel_multi_index}
        1    0.023    0.023    0.023    0.023 {built-in method numpy.core.multiarray.bincount}
        1    0.022    0.022    0.179    0.179 ../zarr/indexing.py:613(__init__)




``.vindex[]`` is a bit slower, possibly because internally it converts to a coordinate array first.

### int dense selection

In [104]:
ix_dense_int = np.random.choice(c.shape[0], size=c.shape[0]//10, replace=True)
ix_dense_int_sorted = ix_dense_int.copy()
ix_dense_int_sorted.sort()
len(ix_dense_int)

10000000

In [105]:
%timeit c[ix_dense_int_sorted]

60.7 ms ± 599 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [106]:
%timeit zc.oindex[ix_dense_int_sorted]

361 ms ± 22.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [107]:
%timeit zc.vindex[ix_dense_int_sorted]

349 ms ± 3.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [108]:
%timeit c[ix_dense_int]

128 ms ± 555 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [109]:
%timeit zc.oindex[ix_dense_int]

1.72 s ± 35 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [110]:
%timeit zc.vindex[ix_dense_int]

1.69 s ± 12.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [118]:
profile('zc.oindex[ix_dense_int_sorted]')

Wed Nov  8 11:19:28 2017    /tmp/tmpk_0eq5a2

         79967 function calls in 0.410 seconds

   Ordered by: internal time
   List reduced from 88 to 7 due to restriction <7>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.146    0.146    0.188    0.188 ../zarr/indexing.py:342(__init__)
     1024    0.093    0.000    0.098    0.000 ../zarr/core.py:1028(_decode_chunk)
     1024    0.045    0.000    0.164    0.000 ../zarr/core.py:849(_chunk_getitem)
     1025    0.025    0.000    0.026    0.000 ../zarr/indexing.py:404(__iter__)
        1    0.023    0.023    0.023    0.023 {built-in method numpy.core.multiarray.bincount}
        1    0.011    0.011    0.011    0.011 /home/aliman/pyenv/zarr_20171023/lib/python3.6/site-packages/numpy/lib/function_base.py:1848(diff)
     1025    0.006    0.000    0.052    0.000 ../zarr/indexing.py:544(__iter__)




In [119]:
profile('zc.vindex[ix_dense_int_sorted]')

Wed Nov  8 11:19:31 2017    /tmp/tmpzhzjc9l7

         51336 function calls in 0.384 seconds

   Ordered by: internal time
   List reduced from 84 to 7 due to restriction <7>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.107    0.054    0.107    0.054 ../zarr/indexing.py:640(<genexpr>)
     1024    0.090    0.000    0.095    0.000 ../zarr/core.py:1028(_decode_chunk)
     1024    0.043    0.000    0.160    0.000 ../zarr/core.py:849(_chunk_getitem)
        1    0.029    0.029    0.029    0.029 {built-in method numpy.core.multiarray.ravel_multi_index}
        1    0.027    0.027    0.199    0.199 ../zarr/indexing.py:613(__init__)
        1    0.023    0.023    0.023    0.023 {built-in method numpy.core.multiarray.bincount}
     2048    0.011    0.000    0.011    0.000 ../zarr/indexing.py:705(<genexpr>)




In [120]:
profile('zc.oindex[ix_dense_int]')

Wed Nov  8 11:19:35 2017    /tmp/tmp52ytv9go

         98407 function calls in 1.780 seconds

   Ordered by: internal time
   List reduced from 91 to 7 due to restriction <7>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    1.113    1.113    1.113    1.113 {method 'argsort' of 'numpy.ndarray' objects}
     1024    0.149    0.000    0.301    0.000 ../zarr/core.py:849(_chunk_getitem)
        1    0.129    0.129    1.404    1.404 ../zarr/indexing.py:342(__init__)
     1024    0.123    0.000    0.130    0.000 ../zarr/core.py:1028(_decode_chunk)
        1    0.121    0.121    0.121    0.121 {method 'take' of 'numpy.ndarray' objects}
     1025    0.031    0.000    0.032    0.000 ../zarr/indexing.py:404(__iter__)
        1    0.023    0.023    0.023    0.023 {built-in method numpy.core.multiarray.bincount}




In [121]:
profile('zc.vindex[ix_dense_int]')

Wed Nov  8 11:19:41 2017    /tmp/tmpmrkck66m

         49294 function calls in 1.738 seconds

   Ordered by: internal time
   List reduced from 86 to 7 due to restriction <7>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    1.112    1.112    1.112    1.112 {method 'argsort' of 'numpy.ndarray' objects}
     1024    0.137    0.000    0.278    0.000 ../zarr/core.py:849(_chunk_getitem)
        2    0.121    0.061    0.121    0.061 ../zarr/indexing.py:664(<genexpr>)
     1024    0.112    0.000    0.119    0.000 ../zarr/core.py:1028(_decode_chunk)
        2    0.106    0.053    0.106    0.053 ../zarr/indexing.py:640(<genexpr>)
        1    0.028    0.028    0.028    0.028 {built-in method numpy.core.multiarray.ravel_multi_index}
        1    0.026    0.026    1.431    1.431 ../zarr/indexing.py:613(__init__)




When indices are not sorted, zarr needs to partially sort them so the occur in chunk order, so we only have to visit each chunk once. This sorting dominates the processing time and is unavoidable AFAIK.

### bool sparse selection

In [122]:
# relatively sparse selection
ix_sparse_bool = np.random.binomial(1, 0.0001, size=c.shape[0]).astype(bool)
np.count_nonzero(ix_sparse_bool)

9985

In [123]:
%timeit c[ix_sparse_bool]

15.6 ms ± 51 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [124]:
%timeit zc.oindex[ix_sparse_bool]

153 ms ± 1.12 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [125]:
%timeit zc.vindex[ix_sparse_bool]

132 ms ± 580 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [126]:
profile('zc.oindex[ix_sparse_bool]')

Wed Nov  8 11:20:47 2017    /tmp/tmpsvc8enk3

         82936 function calls in 0.221 seconds

   Ordered by: internal time
   List reduced from 82 to 7 due to restriction <7>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     1023    0.108    0.000    0.113    0.000 ../zarr/core.py:1028(_decode_chunk)
     1024    0.018    0.000    0.018    0.000 {method 'nonzero' of 'numpy.ndarray' objects}
     1024    0.018    0.000    0.018    0.000 {built-in method numpy.core.multiarray.count_nonzero}
     1023    0.007    0.000    0.145    0.000 ../zarr/core.py:849(_chunk_getitem)
     1024    0.006    0.000    0.052    0.000 ../zarr/indexing.py:544(__iter__)
     1023    0.005    0.000    0.027    0.000 /home/aliman/pyenv/zarr_20171023/lib/python3.6/site-packages/numpy/lib/index_tricks.py:26(ix_)
     2046    0.004    0.000    0.004    0.000 ../zarr/core.py:337(<genexpr>)




In [127]:
profile('zc.vindex[ix_sparse_bool]')

Wed Nov  8 11:20:51 2017    /tmp/tmplmk05u0l

         51304 function calls in 0.171 seconds

   Ordered by: internal time
   List reduced from 87 to 7 due to restriction <7>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     1023    0.099    0.000    0.104    0.000 ../zarr/core.py:1028(_decode_chunk)
        2    0.019    0.010    0.019    0.010 {method 'nonzero' of 'numpy.ndarray' objects}
     1024    0.008    0.000    0.015    0.000 ../zarr/indexing.py:684(__iter__)
     1023    0.007    0.000    0.134    0.000 ../zarr/core.py:849(_chunk_getitem)
     2046    0.005    0.000    0.005    0.000 ../zarr/indexing.py:705(<genexpr>)
     2052    0.003    0.000    0.003    0.000 ../zarr/core.py:337(<genexpr>)
        1    0.002    0.002    0.151    0.151 ../zarr/core.py:591(_get_selection)




### int sparse selection

In [128]:
ix_sparse_int = np.random.choice(c.shape[0], size=c.shape[0]//10000, replace=True)
ix_sparse_int_sorted = ix_sparse_int.copy()
ix_sparse_int_sorted.sort()
len(ix_sparse_int)

10000

In [129]:
%timeit c[ix_sparse_int_sorted]

18.6 µs ± 199 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [130]:
%timeit c[ix_sparse_int]

21.6 µs ± 348 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [131]:
%timeit zc.oindex[ix_sparse_int_sorted]

126 ms ± 2.47 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [132]:
%timeit zc.vindex[ix_sparse_int_sorted]

110 ms ± 483 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [133]:
%timeit zc.oindex[ix_sparse_int]

127 ms ± 652 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [134]:
%timeit zc.vindex[ix_sparse_int]

106 ms ± 332 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [135]:
profile('zc.oindex[ix_sparse_int]')

Wed Nov  8 11:22:21 2017    /tmp/tmp58yngtag

         98407 function calls in 0.198 seconds

   Ordered by: internal time
   List reduced from 91 to 7 due to restriction <7>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     1024    0.104    0.000    0.110    0.000 ../zarr/core.py:1028(_decode_chunk)
     1025    0.012    0.000    0.014    0.000 ../zarr/indexing.py:404(__iter__)
     1025    0.007    0.000    0.053    0.000 ../zarr/indexing.py:544(__iter__)
     2048    0.007    0.000    0.013    0.000 /home/aliman/pyenv/zarr_20171023/lib/python3.6/site-packages/numpy/lib/index_tricks.py:26(ix_)
     1024    0.006    0.000    0.139    0.000 ../zarr/core.py:849(_chunk_getitem)
     2048    0.004    0.000    0.004    0.000 ../zarr/core.py:337(<genexpr>)
    13324    0.004    0.000    0.010    0.000 {built-in method builtins.isinstance}




In [136]:
profile('zc.vindex[ix_sparse_int]')

Wed Nov  8 11:22:27 2017    /tmp/tmpgyi7rp6s

         49294 function calls in 0.157 seconds

   Ordered by: internal time
   List reduced from 86 to 7 due to restriction <7>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     1024    0.103    0.000    0.109    0.000 ../zarr/core.py:1028(_decode_chunk)
     1025    0.009    0.000    0.016    0.000 ../zarr/indexing.py:684(__iter__)
     1024    0.006    0.000    0.137    0.000 ../zarr/core.py:849(_chunk_getitem)
     2048    0.005    0.000    0.005    0.000 ../zarr/indexing.py:705(<genexpr>)
     2054    0.003    0.000    0.003    0.000 ../zarr/core.py:337(<genexpr>)
        1    0.003    0.003    0.156    0.156 ../zarr/core.py:591(_get_selection)
     1024    0.002    0.000    0.005    0.000 /home/aliman/pyenv/zarr_20171023/lib/python3.6/site-packages/numpy/core/arrayprint.py:381(wrapper)




For sparse selections, processing time is dominated by decompression, so we can't do any better.

### sparse bool selection as zarr array

In [137]:
zix_sparse_bool = zarr.array(ix_sparse_bool)
zix_sparse_bool.info

Type,zarr.core.Array
Data type,bool
Shape,"(100000000,)"
Chunk shape,"(390625,)"
Order,C
Read-only,False
Compressor,"Blosc(cname='lz4', clevel=5, shuffle=SHUFFLE, blocksize=0)"
Store type,builtins.dict
No. bytes,100000000 (95.4M)
No. bytes stored,507490 (495.6K)
Storage ratio,197.0


In [138]:
%timeit zc.oindex[zix_sparse_bool]

377 ms ± 12.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### slice with step

In [139]:
%timeit np.array(c[::2])

81 ms ± 969 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [140]:
%timeit np.array(c[::-2])

85.3 ms ± 5.18 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [143]:
%timeit zc.oindex[::2]

1.01 s ± 6.74 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [144]:
%timeit zc.oindex[::-2]

1.18 s ± 5.95 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [145]:
%timeit zc.oindex[::10]

326 ms ± 3.55 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [149]:
%timeit zc.oindex[::-10]

361 ms ± 2.86 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [146]:
%timeit zc.oindex[::100]

152 ms ± 585 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [147]:
%timeit zc.oindex[::1000]

125 ms ± 772 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [148]:
profile('zc.oindex[::2]')

Wed Nov  8 11:24:57 2017    /tmp/tmpodht8l8m

         79942 function calls in 1.075 seconds

   Ordered by: internal time
   List reduced from 82 to 7 due to restriction <7>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.479    0.479    0.595    0.595 ../zarr/indexing.py:342(__init__)
     1024    0.117    0.000    0.256    0.000 ../zarr/core.py:849(_chunk_getitem)
        1    0.116    0.116    0.116    0.116 {built-in method numpy.core.multiarray.bincount}
     1024    0.110    0.000    0.116    0.000 ../zarr/core.py:1028(_decode_chunk)
     1025    0.109    0.000    0.110    0.000 ../zarr/indexing.py:404(__iter__)
        1    0.069    0.069    0.069    0.069 {built-in method numpy.core.multiarray.arange}
        1    0.014    0.014    1.073    1.073 ../zarr/core.py:537(get_orthogonal_selection)




Here there are various setup operations that need to be done on the integer array, can't see way to avoid ATM.

## 2D Benchmarking

In [150]:
c.shape

(100000000,)

In [151]:
d = c.reshape(-1, 1000)
d.shape

(100000, 1000)

In [152]:
zd = zarr.array(d)
zd.info

Type,zarr.core.Array
Data type,int64
Shape,"(100000, 1000)"
Chunk shape,"(3125, 32)"
Order,C
Read-only,False
Compressor,"Blosc(cname='lz4', clevel=5, shuffle=SHUFFLE, blocksize=0)"
Store type,builtins.dict
No. bytes,800000000 (762.9M)
No. bytes stored,39228864 (37.4M)
Storage ratio,20.4


### bool orthogonal selection

In [153]:
ix0 = np.random.binomial(1, 0.5, size=d.shape[0]).astype(bool)
ix1 = np.random.binomial(1, 0.5, size=d.shape[1]).astype(bool)

In [154]:
%timeit d[np.ix_(ix0, ix1)]

98.2 ms ± 995 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [155]:
%timeit zd.oindex[ix0, ix1]

362 ms ± 5.78 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### int orthogonal selection

In [156]:
ix0 = np.random.choice(d.shape[0], size=int(d.shape[0] * .5), replace=True)
ix1 = np.random.choice(d.shape[1], size=int(d.shape[1] * .5), replace=True)

In [157]:
%timeit d[np.ix_(ix0, ix1)]

176 ms ± 2.48 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [158]:
%timeit zd.oindex[ix0, ix1]

550 ms ± 9.16 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### coordinate (point) selection

In [159]:
n = int(d.size * .1)
ix0 = np.random.choice(d.shape[0], size=n, replace=True)
ix1 = np.random.choice(d.shape[1], size=n, replace=True)
n

10000000

In [160]:
%timeit d[ix0, ix1]

241 ms ± 1.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [161]:
%timeit zd.vindex[ix0, ix1]

2.02 s ± 7.81 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [162]:
profile('zd.vindex[ix0, ix1]')

Wed Nov  8 11:28:12 2017    /tmp/tmp2r3kxkv1

         61645 function calls in 2.260 seconds

   Ordered by: internal time
   List reduced from 88 to 7 due to restriction <7>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    1.164    1.164    1.164    1.164 {method 'argsort' of 'numpy.ndarray' objects}
        3    0.272    0.091    0.272    0.091 ../zarr/indexing.py:664(<genexpr>)
        3    0.202    0.067    0.202    0.067 ../zarr/indexing.py:640(<genexpr>)
     1024    0.201    0.000    0.433    0.000 ../zarr/core.py:849(_chunk_getitem)
     1024    0.187    0.000    0.197    0.000 ../zarr/core.py:1028(_decode_chunk)
        1    0.056    0.056    1.778    1.778 ../zarr/indexing.py:613(__init__)
        1    0.044    0.044    0.044    0.044 {built-in method numpy.core.multiarray.ravel_multi_index}




Points need to be partially sorted so all points in the same chunk are grouped and processed together. This requires ``argsort`` which dominates time.

## h5py comparison

N.B., not really fair because using slower compressor, but for interest...

In [65]:
import h5py
import tempfile

In [78]:
h5f = h5py.File(tempfile.mktemp(), driver='core', backing_store=False)

In [79]:
hc = h5f.create_dataset('c', data=c, compression='gzip', compression_opts=1, chunks=zc.chunks, shuffle=True)
hc

<HDF5 dataset "c": shape (100000000,), type "<i8">

In [80]:
%time hc[:]

CPU times: user 1.16 s, sys: 172 ms, total: 1.33 s
Wall time: 1.32 s


array([       0,        1,        2, ..., 99999997, 99999998, 99999999])

In [81]:
%time hc[ix_sparse_bool]

CPU times: user 1.11 s, sys: 0 ns, total: 1.11 s
Wall time: 1.11 s


array([    1063,    28396,    37229, ..., 99955875, 99979354, 99995791])

In [82]:
# # this is pathological, takes minutes 
# %time hc[ix_dense_bool]

In [83]:
# this is pretty slow
%time hc[::1000]

CPU times: user 38.3 s, sys: 136 ms, total: 38.4 s
Wall time: 38.1 s


array([       0,     1000,     2000, ..., 99997000, 99998000, 99999000])